In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
folder_path = "/content/drive/MyDrive/Colab Notebooks/study/detr_colab"
import sys
sys.path.insert(0, folder_path)

In [3]:
from scipy.optimize import linear_sum_assignment
from models.matcher import HungarianMatcher
import torch

In [4]:
n_queries, n_targets, n_classes = 100, 15, 91
logits = torch.rand(1, n_queries, n_classes + 1)
boxes = torch.rand(1, n_queries, 4)
tgt_labels = torch.randint(high=n_classes, size=(n_targets,))
tgt_boxes = torch.rand(n_targets, 4)
matcher = HungarianMatcher()
targets = [{'labels': tgt_labels, 'boxes': tgt_boxes}]
indices_single = matcher({'pred_logits': logits, 'pred_boxes': boxes}, targets)
# indices_batched = matcher({'pred_logits': logits.repeat(2, 1, 1),'pred_boxes': boxes.repeat(2, 1, 1)}, targets * 2)

# test with empty targets
# tgt_labels_empty = torch.randint(high=n_classes, size=(0,))
# tgt_boxes_empty = torch.rand(0, 4)
# targets_empty = [{'labels': tgt_labels_empty, 'boxes': tgt_boxes_empty}]
# indices = matcher({'pred_logits': logits.repeat(2, 1, 1), 'pred_boxes': boxes.repeat(2, 1, 1)}, targets + targets_empty)
# indices = matcher({'pred_logits': logits.repeat(2, 1, 1), 'pred_boxes': boxes.repeat(2, 1, 1)}, targets_empty * 2)

cost_class torch.Size([100, 15])
cost_bbox torch.Size([100, 15])
cost_giou torch.Size([100, 15])
C torch.Size([1, 100, 15])


In [5]:
print(indices_single)

[(tensor([ 3,  6, 19, 26, 40, 41, 44, 47, 49, 50, 52, 54, 59, 77, 90]), tensor([ 8,  6, 13,  9,  1,  0,  7, 10, 11,  4,  5, 14,  3, 12,  2]))]


In [6]:
pred = [[
    [0.1, 0.1, 0.7, 0.05, 0.05],
    [0.9, 0.05, 0.0, 0.0, 0.05],
    [0.3, 0.1, 0.2, 0.2, 0.2],
    [0.1, 0.0, 0.0, 0.1, 0.8],
    [0.5, 0.2, 0.0, 0.0, 0.3]
]]

label = [[0,2,4]]

pred = torch.tensor(pred)
label = torch.tensor(label)

In [7]:
pred_prob = pred.flatten(0, 1).softmax(-1)

print(pred_prob)

pred_candidate = -pred_prob[:, label]

print(pred_candidate)

tensor([[0.1747, 0.1747, 0.3183, 0.1662, 0.1662],
        [0.3748, 0.1602, 0.1524, 0.1524, 0.1602],
        [0.2206, 0.1806, 0.1996, 0.1996, 0.1996],
        [0.1717, 0.1554, 0.1554, 0.1717, 0.3458],
        [0.2651, 0.1964, 0.1608, 0.1608, 0.2170]])
tensor([[[-0.1747, -0.3183, -0.1662]],

        [[-0.3748, -0.1524, -0.1602]],

        [[-0.2206, -0.1996, -0.1996]],

        [[-0.1717, -0.1554, -0.3458]],

        [[-0.2651, -0.1608, -0.2170]]])


In [9]:
matched_pred, matched_label = linear_sum_assignment(pred_candidate.squeeze(1))
print(matched_pred)
print(matched_label)

[0 1 3]
[1 0 2]
